In [ ]:
## Nível -1 à 4 - TRIAGEM DE E-MAILS

# INTEGRAÇAO DE E-MAIL
from imap_tools import MailBox, AND, MailMessageFlags
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pyautogui
from selenium import webdriver
import mysql.connector
import time
import smtplib
from email.mime.text import MIMEText  
from datetime import datetime

# Acesso e-mail
login = "your login"
senha = "your password"

# Realizar Login
meu_email = MailBox("imap.gmail.com").login(login, senha)

#Todo o código deve ficar identado nesse loop


# Condições de busca de e-mails
lista_emails = meu_email.fetch(criteria=AND(seen=False, from_="any email"),
                    mark_seen=False,
                    bulk=True)

if lista_emails != None:

    # Busca de ID's no pipe 
    pipe_ids = [] 

    for email in lista_emails: 
        pipe_id = email.html
        pipe_id = bs(email.html)
        pipe_id = pipe_id.find("a")["href"]
        position = pipe_id.find("id=")
        pipe_id = pipe_id[position +3:]
        pipe_ids.append(pipe_id)
        print(pipe_ids)


        # Conect MySQL DataBase to Python
        connection = mysql.connector.connect(host='some.host.com',
                                                         database='some',
                                                         user='some',
                                                         password='some')

        mycursor = connection.cursor()
        sql = "SELECT * FROM briefings WHERE id={}".format(pipe_id)
        mycursor.execute(sql)
        resultado = mycursor.fetchone()
        nomeFantasia = resultado[1]   # Nome Fantasia
        emaill = resultado[2] 
        responsavel = resultado[4]   # Responsável
        endereco = resultado[5]   # Endereço
        telefone = resultado[6]   # Telefone
        site = resultado[7]   # Site
        servico = resultado[8]   # Serviço (Ele junta os dois veiculos em um campo)
        valorMensal = resultado[9]   # Valor mensal
        cooparticipacao = resultado[12]  # Coparticipação
        idGoogleAds = resultado[36]  # ID Google Ads
        
        porcentagem = (cooparticipacao / 100)
        participacao_digi = (valorMensal * porcentagem)

        #Verificação se o briefing possui ID google ads para prosseguir com automação
        if idGoogleAds is None:
            if participacao_digi >= 1000:

                sender = 'any sender'
                receiver = 'any receiver'

                body = "ID Conta Google Ads: {}\nCliente: {}\nMídia Contratada: {}\nValor Investido: {}\nCoparticipacao: {} %\nSite do Cliente: {}\nObservações Importantes: ".format(idGoogleAds, nomeFantasia, servico, valorMensal, cooparticipacao, site)
                msg = MIMEText(body)
                msg['Subject'] = 'Cliente Magnum: {} - ID Conta Google Ads não encontrado '.format(nomeFantasia)
                msg['From'] = 'any sender'
                msg['To'] = 'any receiver'

                user = 'your login'
                password = 'your password'

                with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:

                    server.login(user, password)
                    server.sendmail(sender, receiver, msg.as_string())
                    print("mail successfully sent")
                    meu_email.flag(email.uid, MailMessageFlags.SEEN, True)
                
            else:
                sender = 'any sender'
                receiver = 'any receiver'
                body = "ID Conta Google Ads: {}\nNome do Cliente: {}\nMídia Contratada: {}\nValor Investido: {}\nCoparticipacao: {} %\nSite do Cliente: {}".format(idGoogleAds, nomeFantasia, servico, valorMensal, cooparticipacao, site)
                msg = MIMEText(body)
                msg['Subject'] = 'Cliente Standard: {} - ID Conta Google Ads não encontrado '.format(nomeFantasia)
                msg['From'] = 'your login'
                msg['To'] = 'your password'

                user = 'your login'
                password = 'your receiver'

                with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:

                    server.login(user, password)
                    server.sendmail(sender, receiver, msg.as_string())
                    print("mail successfully sent")
                    
                    meu_email.flag(email.uid, MailMessageFlags.SEEN, True)
        else:            
        # Acessar Google Ads na útlima URL possível
            driver = webdriver.Chrome()
            url = driver.get("https://ads.google.com/aw/settings/subaccounts?ocid=58110198&euid=587467136&__u=1668078464&uscid=58110198&__c=2221948102&authuser=0&subid=ww-ww-et-g-aw-a-vasquette_ads_cons_1%21o2")

            # Fazer Login
            time.sleep(2)
            username = driver.find_element_by_id("identifierId")
            username.send_keys("your login", Keys.ENTER)

            time.sleep(2)
            password = driver.find_element_by_name("password")
            password.send_keys("your password", Keys.ENTER)

            # Click no botão +
            time.sleep(20)
            pyautogui.click (x=257, y=677)

            # Click no botão p/ add conta
            time.sleep(5)
            pyautogui.click (x=368, y=845)

            # Copiar váriavel
            time.sleep(5)
            pyautogui.typewrite(idGoogleAds)  
            time.sleep(5)
            #Botão enviar convite
            pyautogui.click(x=337, y=733)
            time.sleep(5)
            #Confirmar identidade
            pyautogui.click(x=1062, y=696)
            time.sleep(5)


            if participacao_digi >= 1000: 

                sender = 'any sender'
                receiver = (email)

                body = "ID Conta Google Ads: {}\nCliente: {}\nMídia Contratada: {}\nValor Investido: {}\nCoparticipacao: {} %\nSite do Cliente: {}\nObservações Importantes: ".format(idGoogleAds, nomeFantasia, servico, valorMensal, cooparticipacao, site)
                msg = MIMEText(body)

                msg['Subject'] = 'Cliente Magnum: {} - Convite de acesso à conta Google Ads Enviado '.format(nomeFantasia)
                msg['From'] = 'any sender'
                msg['To'] = 'any receiver'

                user = 'your email'
                password = 'your password'

                with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:

                    server.login(user, password)
                    server.sendmail(sender, receiver, msg.as_string())
                    print("mail successfully sent")
                    meu_email.flag(email.uid, MailMessageFlags.SEEN, True)
            else:

                sender = 'any sender'
                receiver = 'any receiver'

                body = "ID Conta Google Ads: {}\nNome do Cliente: {}\nMídia Contratada: {}\nValor Investido: {}\nCoparticipacao: {} %\nSite do Cliente: {}".format(idGoogleAds, nomeFantasia, servico, valorMensal, cooparticipacao, site)
                msg = MIMEText(body)

                msg['Subject'] = 'Cliente Standard: {} - Convite de acesso à conta Google Ads Enviado '.format(nomeFantasia)
                msg['From'] = 'any sender'
                msg['To'] = 'any receiver'

                user = 'your login'
                password = 'your password'

                with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:

                    server.login(user, password)
                    server.sendmail(sender, receiver, msg.as_string())
                    print("mail successfully sent")
                    meu_email.flag(email.uid, MailMessageFlags.SEEN, True)
